In [1]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import sympy as sy

# Part I - Symbolic Problems

## Q1

In [2]:
n = sp.symbols('n', integer=True, positive=True)
F = sp.MatrixSymbol('F', n, n)           # arbitrary invertible tensor (matrix)
I = sp.Identity(n)

# Right Cauchy-Green Tensor
C = F.T @ F
# Left Cauchy-Green Tensor
B = F @ F.T

# Right symmetric tensor
U = sp.sqrt(C)
# Left symmetric tensor
V = sp.sqrt(B)

# Isolate Q from right polar system
Q_right = F @ sp.Inverse(U)
# Isolate Q from left polar system
Q_left = sp.Inverse(V) @ F

# Polar decompositions (formal equalities)
eq1 = sp.Eq(F, Q_right * U)               # F = Q U
eq2 = sp.Eq(F, V * Q_left)                # F = V Q

if eq1 == eq2 == True:
    print(f"F = VQ = QU")



############## Right Polar Decomposition ##############
symU = sp.Eq(U, U.T)                      # U is symmetric
# substitute using the assumptions:
# 1) U is symmetric: Inverse(U).T -> Inverse(U)
# 2) F.T*F = U*U  (i.e. C = U^2)
# 3) Inverse(U)*U = I and U*Inverse(U) = I
U_sy = sp.MatrixSymbol('U', n, n)
Q_right_sy = F @ sp.Inverse(U_sy)
expr_test_Q_orthogonality_right = Q_right_sy.T @ Q_right_sy
expr_test_Q_orthogonality_right = expr_test_Q_orthogonality_right.subs(sp.Inverse(U_sy).T, sp.Inverse(U_sy))
expr_test_Q_orthogonality_right = expr_test_Q_orthogonality_right.subs(F.T * F, U_sy * U_sy)
print("############## Right Polar Decomposition ##############")
print("U symmetric? ", symU )
print("After substitutions (should be Identity): ", expr_test_Q_orthogonality_right)
print("QT · Q = Q−1 · Q = I (Q is orthogonal)")
UU   = sp.Eq(U*U, C)                      # U^2 = F^T F
if UU:
    print("UU = F.T F")

############## Left Polar Decomposition ##############
symV = sp.Eq(V, V.T)                      # V is symmetric
# substitute using the assumptions:
# 1) V is symmetric: Inverse(V).T -> Inverse(V)
# 2)   (i.e. B = V^2)
# 3) Inverse(V)*V = I and V*Inverse(V) = I
V_sy = sp.MatrixSymbol('V', n, n)
Q_left_sy = sp.Inverse(V_sy) @ F
expr_test_Q_orthogonality_left = Q_left_sy @ Q_left_sy.T
expr_test_Q_orthogonality_left = expr_test_Q_orthogonality_left.subs(sp.Inverse(V_sy).T, sp.Inverse(V_sy))
expr_test_Q_orthogonality_left = expr_test_Q_orthogonality_left.subs(F * F.T, V_sy * V_sy)
print("############## Left Polar Decomposition ##############")
print("V symmetric? ", symV )
print("After substitutions (should be Identity): ", expr_test_Q_orthogonality_left)
VV   = sp.Eq(V*V, B)                      # V^2 = F F^T
if VV:
    print("VV = F F.T")


F = VQ = QU
############## Right Polar Decomposition ##############
U symmetric?  True
After substitutions (should be Identity):  I
QT · Q = Q−1 · Q = I (Q is orthogonal)
UU = F.T F
############## Left Polar Decomposition ##############
V symmetric?  True
After substitutions (should be Identity):  I
VV = F F.T


## Q2

In [3]:
n = sp.symbols('n', integer=True, positive=True)

# Define symbolic matrices
C = sp.MatrixSymbol('C', n, n)
D = sp.MatrixSymbol('D', n, n)

# Assumptions
# C symmetric: C.T = C
# D skew-symmetric: D.T = -D

# Trace of C*D
trace_CD = sp.Trace(C @ D)

# Use properties: Trace(A) = Trace(A.T), C^T = C, D^T = -D
trace_T_CD = sp.Trace((C @ D).T)
trace_T_CD = trace_T_CD.subs(C.T, C).subs(D.T, -D)

# Use property Trace(CD)=Trace(DC)
# trace_T_CD = sp.simplify(trace_T_CD.subs(sp.Trace(D@C), sp.Trace(C@D)))
trace_T_CD = sp.simplify(trace_T_CD).subs(sp.Trace(D@C), sp.Trace(C@D))


# Solve / simplify
trace_simplified = sp.simplify(trace_CD + trace_T_CD)
print("Trace(C*D) = 0 check:", trace_simplified == 0,  "\n(if C is symmetric and D is skew-symmetric)")

Trace(C*D) = 0 check: True 
(if C is symmetric and D is skew-symmetric)


## Q3

In [4]:
n = 3  # 3D
C = sp.MatrixSymbol('C', n, n)
I = sp.Identity(n)

# Deviatoric part: dev(C) = C - (1/3)*tr(C)*I
devC = C - (1/3) * sp.Trace(C) * I


# First invariant: I1(devC) = tr(dev(C))
I1_devC = sp.Trace(devC).doit()
print("I1(devC) =", I1_devC)

# Second invariant: I2(devC) = -1/2 * tr(dev(C)^2), as tr(dev(C))=0
I2_devC = 0.5 * ( - sp.Trace(devC**2)).doit()
print("I2(devC) =", I2_devC,"=-0.5*Trace(((dev(C))**2)")

# Third invariant (general formula for det(A) in 3D):
I3_devC = sp.Rational(1, 6) * (
    (sp.Trace(devC))**3
    - 3*sp.Trace(devC)*sp.Trace(devC**2)
    + 2*sp.Trace(devC**3)
).doit()
print("I3(devC) =", sp.simplify(I3_devC),"=-1/3*Trace(((dev(C))**3)")

I1(devC) = 0
I2(devC) = -0.5*Trace(((-0.333333333333333*Trace(C))*I + C)**2) =-0.5*Trace(((dev(C))**2)
I3(devC) = Trace(((-0.333333333333333*Trace(C))*I + C)**3)/3 =-1/3*Trace(((dev(C))**3)


# Part II - Numerical Problems

In [5]:
T = np.array([[1, 0, 0],
              [0, 1.25, -np.sqrt(3)/4],
              [0, -np.sqrt(3)/4, 9/4]])

A = np.array([[1,2,3],
              [4,2,1],
              [1,1,1]])

def Rotation(alpha, beta, gamma):
    def Rx(theta):
        return np.array([[1, 0, 0],
                         [0, np.cos(theta), -np.sin(theta)],
                         [0, np.sin(theta), np.cos(theta)]])
    def Ry(theta):
        return np.array([[np.cos(theta), 0, np.sin(theta)],
                         [0, 1, 0],
                         [-np.sin(theta), 0, np.cos(theta)]])
    def Rz(theta):
        return np.array([[np.cos(theta), -np.sin(theta), 0],
                         [np.sin(theta), np.cos(theta), 0],
                         [0, 0, 1]])
    return Rz(alpha) @ Ry(beta) @ Rx(gamma)

## Q1

### a) Setup the original base vectors

In [6]:
e1_cartesian = np.array([1, 0, 0]).reshape(-1,1)
e2_cartesian = np.array([0, 1, 0]).reshape(-1,1)
e3_cartesian = np.array([0, 0, 1]).reshape(-1,1)
print("e1 in the 3D Cartesian system is: \n", e1_cartesian)
print("e2 in the 3D Cartesian system is: \n", e2_cartesian)
print("e3 in the 3D Cartesian system is: \n", e3_cartesian)

e1 in the 3D Cartesian system is: 
 [[1]
 [0]
 [0]]
e2 in the 3D Cartesian system is: 
 [[0]
 [1]
 [0]]
e3 in the 3D Cartesian system is: 
 [[0]
 [0]
 [1]]


### b-c) Rotate the base vectors with given $\alpha$, $\beta$ and $\gamma$
Compute the corresponding Q matrix 

In [ ]:
alpha_deg = 30
beta_deg = 15
gamma_deg = 25

deg_to_rad = lambda theta: theta * np.pi / 180

R = Rotation(alpha=deg_to_rad(alpha_deg),
             beta=deg_to_rad(beta_deg),
             gamma=deg_to_rad(gamma_deg))

e1_rotated = R @ e1_cartesian
e2_rotated = R @ e2_cartesian
e3_rotated = R @ e3_cartesian

print(f"With the given rotation angles, obtained rotation tensor is: \n {R}")
print(f"The determinant of the rotation tensor is {np.linalg.det(R):.3f} \n\
which tells us Q = R is proper orthogonal thus: it is the Rotation Tensor \n\
and the new basis is also right handed!")

print("e1 in the rotated system is: \n", e1_rotated)
print("e2 in the rotated system is: \n", e2_rotated)
print("e3 in the rotated system is: \n", e3_rotated)

print(f"Norm of new e1 is {np.linalg.norm(e1_rotated)}")
print(f"Norm of new e2 is {np.linalg.norm(e2_rotated):.3f}")
print(f"Norm of new e3 is {np.linalg.norm(e3_rotated)}")
print("Thus the new sets of base vectors are orthonormal!")


print("Also check for Q.T @ Q = Q @ Q.T = I")

QTQ = R.T @ R
QQT = R @ R.T

print("Is Q^T Q identity?", np.allclose(QTQ, np.eye(3)))
print("Is Q Q^T identity?", np.allclose(QQT, np.eye(3)))



With the given rotation angles, obtained rotation tensor is: 
 [[ 0.8365163  -0.3584266   0.41445246]
 [ 0.48296291  0.83957639 -0.24871329]
 [-0.25881905  0.40821789  0.8754261 ]]
The determinant of the rotation tensor is 1.000 
which tells us Q = R is proper orthogonal thus: it is the Rotation Tensor 
and the new basis is also right handed!
e1 in the rotated system is: 
 [[ 0.8365163 ]
 [ 0.48296291]
 [-0.25881905]]
e2 in the rotated system is: 
 [[-0.3584266 ]
 [ 0.83957639]
 [ 0.40821789]]
e3 in the rotated system is: 
 [[ 0.41445246]
 [-0.24871329]
 [ 0.8754261 ]]
Norm of new e1 is 1.0
Norm of new e2 is 1.000
Norm of new e3 is 1.0
Thus the new sets of base vectors are orthonormal!
Also check for Q.T @ Q = Q @ Q.T = I
Is Q^T Q identity? True
Is Q Q^T identity? True


### d) Compute $T$′

In [50]:
compute_transform_tensor_T = lambda T, Q: Q @ T @ Q.T
T_prime = compute_transform_tensor_T(T=T,Q=R)

print(f"T prime is: \n {T_prime}")

T prime is: 
 [[ 1.37547972 -0.39335559  0.479558  ]
 [-0.39335559  1.43438307 -0.46077541]
 [ 0.479558   -0.46077541  1.69013721]]


### e) Compute the eigenvalues of $T$′

In [51]:
eigvals_T, eigvecs_T = np.linalg.eig(T)
eigvals_T_prime, eigvecs_T_prime = np.linalg.eig(T_prime)

eigvals_T_inds = np.argsort(eigvals_T, )[::-1]
eigvals_T_sorted = eigvals_T[eigvals_T_inds]

eigvals_T_prime_inds = np.argsort(eigvals_T_prime, )[::-1]
eigvals_T_prime_sorted = eigvals_T_prime[eigvals_T_prime_inds]


print('Sorted eigenvalues of T:')
print(eigvals_T_sorted)
print('Sorted eigenvectors of T prime')
print(eigvals_T_prime_sorted)
print(f"Are T and T prime have the same eigenvalue? {np.allclose(eigvals_T_sorted, eigvals_T_prime_sorted)}")


Sorted eigenvalues of T:
[2.41143783 1.08856217 1.        ]
Sorted eigenvectors of T prime
[2.41143783 1.08856217 1.        ]
Are T and T prime have the same eigenvalue? True


## Q2

In [59]:
find_first_invariant = lambda C: np.trace(C)
find_second_invariant = lambda C: 0.5 * ((np.trace(C)**2) - np.trace(C @ C))
find_third_invariant = lambda C: np.linalg.det(C)

first_variant_T = find_first_invariant(T)
second_variant_T = find_second_invariant(T)
third_variant_T = find_third_invariant(T)

first_variant_T_prime = find_first_invariant(T_prime)
second_variant_T_prime = find_second_invariant(T_prime)
third_variant_T_prime = find_third_invariant(T_prime)



print(f"The first invariant of the T is: {first_variant_T}")
print(f"The second invariant of the T is: {second_variant_T}")
print(f"The third invariant of the T is: {third_variant_T}")

print(f"The first invariant of the T prime is: {first_variant_T_prime}")
print(f"The second invariant of the T prime is: {second_variant_T_prime}")
print(f"The third invariant of the T prime is: {third_variant_T_prime}")


print(f"Are the first invariants the same: {np.isclose(first_variant_T, first_variant_T_prime)}")
print(f"Are the second invariants the same: {np.isclose(second_variant_T, second_variant_T_prime)}")
print(f"Are the third invariants the same: {np.isclose(third_variant_T, third_variant_T_prime)}")

check_characteristic_cubic_equation = lambda I_T, II_T, III_T, eigval: eigval**3 - I_T * eigval**2 + II_T * eigval - III_T
cayley_ham_T = check_characteristic_cubic_equation(first_variant_T, 
                                    second_variant_T,
                                    third_variant_T,
                                    eigvals_T_sorted) 

cayley_ham_T_prime = check_characteristic_cubic_equation(first_variant_T_prime, 
                                    second_variant_T_prime,
                                    third_variant_T_prime,
                                    eigvals_T_prime_sorted) 

print(f"Is Cayley-Hamilton equation satisfied for T? {np.allclose(cayley_ham_T, np.zeros(3))}")
print(f"Is Cayley-Hamilton equation satisfied for T prime? {np.allclose(cayley_ham_T_prime, np.zeros(3))}")




The first invariant of the T is: 4.5
The second invariant of the T is: 6.125
The third invariant of the T is: 2.625
The first invariant of the T prime is: 4.499999999999999
The second invariant of the T prime is: 6.124999999999997
The third invariant of the T prime is: 2.6249999999999996
Are the first invariants the same: True
Are the second invariants the same: True
Are the third invariants the same: True
Is Cayley-Hamilton equation satisfied for T? True
Is Cayley-Hamilton equation satisfied for T prime? True


## Q3

### a) Decompose the tensor $A$ into the symmetric and an antisymmetric parts

In [ ]:
find_symmetric_part = lambda A: (A + A.T) / 2
find_antisymmetric_part = lambda A: (A - A.T) / 2

A_sym = find_symmetric_part(A)
A_antisym = find_antisymmetric_part(A)

print(f"Symmetric part of the tensor A is: \n {A_sym}")
print(f"Antisymmetric part of the tensor A is: \n {A_antisym}")

Symmetric part of the tensor A is: 
 [[1. 3. 2.]
 [3. 2. 1.]
 [2. 1. 1.]]
Antisymmetric part of the tensor A is: 
 [[ 0. -1.  1.]
 [ 1.  0.  0.]
 [-1.  0.  0.]]


### c) Apply this principal to your answer for (a)

In [65]:
print("Lets denote the symmetric part of the tensor A as C")
C = A_sym
print("Lets denote the antisymmetric part of the tensor A as D")
D = A_antisym
print(f"Trace of the C@D is {np.trace(C @ D)}")


Lets denote the symmetric part of the tensor A as C
Lets denote the antisymmetric part of the tensor A as D
Trace of the C@D is 0.0


## Q4

In [79]:
decompose_into_spherical = lambda C: 1 / 3 * np.trace(C) * np.eye(3)
decompose_into_deviatoric = lambda C: C - decompose_into_spherical(C)

spherical_T = decompose_into_spherical(T)
deviatoric_T = decompose_into_deviatoric(T)

spherical_A = decompose_into_spherical(A)
deviatoric_A = decompose_into_deviatoric(A)

print(f"Spherical part of T is: \n {spherical_T}")
print(f"Deviatoric part of T is: \n {deviatoric_T}")
print(f"Spherical part of A is: \n {spherical_A}")
print(f"Deviatoric part of A is: \n {deviatoric_A}")


trace_dev_T = np.trace(deviatoric_T)
trace_dev_A = np.trace(deviatoric_A)

print(f"Is the trace of dev(T)=0? {np.isclose(trace_dev_T,0)}")
print(f"Is the trace of dev(A)=0? {np.isclose(trace_dev_A,0)}")



Spherical part of T is: 
 [[1.5 0.  0. ]
 [0.  1.5 0. ]
 [0.  0.  1.5]]
Deviatoric part of T is: 
 [[-0.5        0.         0.       ]
 [ 0.        -0.25      -0.4330127]
 [ 0.        -0.4330127  0.75     ]]
Spherical part of A is: 
 [[1.33333333 0.         0.        ]
 [0.         1.33333333 0.        ]
 [0.         0.         1.33333333]]
Deviatoric part of A is: 
 [[-0.33333333  2.          3.        ]
 [ 4.          0.66666667  1.        ]
 [ 1.          1.         -0.33333333]]
Is the trace of dev(T)=0? True
Is the trace of dev(A)=0? True


## Q5

In [80]:
# angle in radians
theta = np.pi / 6   # 30 degrees for example

# Define A (rotation in xy-plane)
A = np.array([
    [np.cos(theta),  np.sin(theta), 0],
    [-np.sin(theta), np.cos(theta), 0],
    [0, 0, 1]
])

# Define B (reflection in x-y plane across line x=-y?)
B = np.array([
    [-1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
])

# Check orthogonality
print("A^T A =\n", A.T @ A)
print("Is A orthogonal?", np.allclose(A.T @ A, np.eye(3)))

print("B^T B =\n", B.T @ B)
print("Is B orthogonal?", np.allclose(B.T @ B, np.eye(3)))

# Determinants
print("det(A) =", np.linalg.det(A))
print("det(B) =", np.linalg.det(B))

# Classification
if np.isclose(np.linalg.det(A), 1.0):
    print("A is a proper orthogonal tensor → a rotation.")
if np.isclose(np.linalg.det(B), -1.0):
    print("B is an improper orthogonal tensor → a reflection.")

A^T A =
 [[ 1.00000000e+00 -7.43708407e-18  0.00000000e+00]
 [-7.43708407e-18  1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Is A orthogonal? True
B^T B =
 [[1 0 0]
 [0 1 0]
 [0 0 1]]
Is B orthogonal? True
det(A) = 1.0
det(B) = -1.0
A is a proper orthogonal tensor → a rotation.
B is an improper orthogonal tensor → a reflection.


# Part III - Hard Tissue Physiology

## Q6) Answer the following in 1-2 paragraphs for each question

### a) What are the major bone types and components of bone?
Bones are classified into four major types based on shape: long bones (e.g., femur, humerus), short bones (e.g., carpals), flat bones (e.g., skull, sternum), and irregular bones (e.g., vertebrae). Structurally, bone is composed of compact (cortical) bone, which is dense and provides strength, and spongy (trabecular or cancellous) bone, which has a porous architecture that reduces weight while maintaining structural support. The main components of bone include the organic matrix (primarily type I collagen, providing flexibility and tensile strength), inorganic mineral phase (hydroxyapatite crystals, conferring hardness and compressive strength), and water. Additionally, bone contains living cells (osteoblasts, osteoclasts, osteocytes) and marrow spaces for hematopoiesis.

### b) Going from the cell to the whole bone levels, name all major structures
At the smallest level, bone is composed of bone cells (osteoblasts, osteoclasts, osteocytes) embedded within an extracellular matrix. These cells are organized into osteons in cortical bone, which are concentric lamellae around a central Haversian canal containing blood vessels and nerves. In cancellous bone, the structure is arranged into a lattice of trabeculae. At a larger scale, bones are wrapped by the periosteum externally and the endosteum internally. The bone is further organized into epiphyses (ends), metaphyses (transition zones), and diaphyses (shaft) in long bones, containing both cortical and cancellous regions. At the whole organ level, bones integrate into the skeletal system, articulating through joints and connecting to muscles, tendons, and ligaments for mechanical and physiological function.

### c) What are the major bone cell types and their respective functions?
There are four major bone cell types. Osteoblasts are bone-forming cells that synthesize and secrete the organic bone matrix and initiate mineralization. Osteoclasts are large, multinucleated cells responsible for bone resorption through enzymatic and acidic degradation of mineralized matrix. Osteocytes are mature bone cells derived from osteoblasts that become embedded within lacunae; they function as mechanosensors and regulate bone remodeling through signaling. Finally, bone lining cells cover inactive bone surfaces, playing roles in bone maintenance and regulating ion exchange. Together, these cells coordinate bone turnover and adaptation.

### d) What is the difference between bone tissue remodeling vs. modeling?
Bone remodeling is the coupled process of bone resorption by osteoclasts and bone formation by osteoblasts at the same site, replacing old or damaged bone with new tissue to maintain skeletal integrity and mineral homeostasis. This is a lifelong process that occurs in localized cycles. In contrast, bone modeling refers to the process where bone formation and resorption occur on different surfaces or at different sites, leading to changes in bone shape, size, and structure. Modeling is most prominent during growth and development, whereas remodeling predominates in adult bone maintenance and repair.

## Q7) Answer the following in 2-3 paragraphs.

### a)
Bone achieves its physiological functions—support, protection, movement, mineral storage, and hematopoiesis—by adapting its geometry and internal structure to mechanical and metabolic demands. Long bones, for example, have thick cortical bone along the shaft to withstand bending and torsional loads, while their ends are composed of cancellous bone to absorb joint stresses. The distribution of cortical vs. cancellous bone can be locally modified; for example, the femoral neck has a trabecular pattern aligned with principal stress trajectories, optimizing load transfer. Similarly, bone mass distribution can change through remodeling in response to Wolff’s law, reinforcing regions under high stress and resorbing bone where stresses are lower, thereby conserving material while maximizing strength.

### b) 
At the morphological level, bones exhibit adaptations that enhance efficiency in fulfilling their roles. Cortical bone, being dense and strong, resists compressive and bending forces, making it essential in shafts of long bones that act as levers. Trabecular bone, with its porous lattice structure, provides lightweight support and facilitates shock absorption, particularly in vertebrae and epiphyses. On the microscopic level, the arrangement of lamellae in osteons allows resistance to torsion, while osteocyte networks ensure adaptive responses to mechanical loading. Across organizational levels, from collagen fibrils providing tensile strength to macroscopic bone curvature distributing loads, each structural adaptation ensures that bones can withstand diverse mechanical challenges while minimizing weight and metabolic cost.